# Language Models

> Inspired by Jeff Dean's talk, [Exciting Trends in Machine Learning](https://youtu.be/oSCRZkSQ1CE).

In this lesson, we'll apply what we know about neural networks toward the study of **language models**: probabilistic models of a natural (human) language, such as English. The goal of today is to become more conversational in language modeling techniques by studying recent natural language processing methods (rather than applications or implications). By the end of this lesson, students will be able to:

- Explain how unigram and bigram language models determine the probability of the next word.
- Explain how embedding spaces can provide a semantic, distributed representation for concepts.
- Explain the benefits of self-attention mechanisms and hidden states in a RNN.

In [ ]:
import os
import random
import re

from collections import Counter
from typing import Any


def clean(token: str, pattern: re.Pattern[str] = re.compile(r"\W+")) -> str:
    """
    Returns all the characters in the token lowercased and without matches to the given pattern.

    >>> clean("Hello!")
    'hello'
    """
    return pattern.sub("", token.lower())


def sample(frequencies: dict[Any, float], k: int = 1) -> list[Any]:
    """
    Returns a list of k randomly sampled keys from the given frequencies with replacement.

    >>> sample({"test": 1})
    ['test']
    >>> sample({"test": 1}, k=3)
    ['test', 'test', 'test']
    """
    return random.choices(list(frequencies), weights=frequencies.values(), k=k)

## Statistical models

In its simplest form, a language model is similar to the `Document` class that we defined in the *Search* assessment consisting of a term-frequency dictionary. A **unigram language model** guesses the next word based on the term-frequency dictionary alone. In the document, `doggos/doc1.txt`, each unique word appears once, so each word has a term frequency of 1.

In [ ]:
terms = {
    "dogs":     1,
    "are":      1,
    "the":      1,
    "greatest": 1,
    "pets":     1,
}

sample(terms, 20)

Unigram models are simple but not particularly useful. For one, there's no notion of context: each word is sampled entirely independently from every other word. Large language models like ChatGPT combine learn from internet-scale training datasets and consider the preceding words (tokens) when determining the probability of the next word.

A **bigram language model**, for instance, takes the immediately preceding word to determine the probabilities for the next word. We use `<s>` to indicate the start token. Modify this code snippet so that each `prev` term (key) has a dictionary consisting of the number of times the `curr` term appears afterwards.

In [ ]:
terms = {}
path = os.path.join("..", "homework", "small_wiki")
for filename in os.listdir(path):
    if filename.endswith(".html"):
        with open(os.path.join(path, filename)) as f:
            words = ["<s>"] + [clean(word) for word in f.read().split() if clean(word)]
            for prev, curr in zip(words, words[1:]):
                terms[prev] = curr # Change this

terms["<s>"] # Should be {"metadata": 70} to indicate all 70 documents start with "metadata"

To generate a sequence of a given length, repeatedly use the `last` word to sample the next word and append it to the result.

In [ ]:
n_words = 20
result = "what is the best".split()
for _ in range(n_words):
    last = result[-1]
    result += sample(terms[last])

result

## Word embeddings

Unigram and bigram language models are more generally known as **n-gram language models**. With larger context windows (n), n-gram models can produce more understandable results. But the approach has a fundamental limitation: it's sensitive to the exact words and the sequence they appeared in the training set.

To address the first problem of learning word meaning, how might a computer even learn the meaning of a word? Strings in Python are sequences of characters where each character is just a number. Or, if you recall how we handled the city names "NY" and "SF" to predict the location of a home in model evaluation, strings could also be represented as "dummy variables" or boolean categories.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>beds</th>
      <th>bath</th>
      <th>year_built</th>
      <th>sqft</th>
      <th>price_per_sqft</th>
      <th>elevation</th>
      <th>city_NY</th>
      <th>city_SF</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2.0</td>
      <td>1.0</td>
      <td>1960</td>
      <td>1000</td>
      <td>999</td>
      <td>10</td>
      <td>True</td>
      <td>False</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2.0</td>
      <td>2.0</td>
      <td>2006</td>
      <td>1418</td>
      <td>1939</td>
      <td>0</td>
      <td>True</td>
      <td>False</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2.0</td>
      <td>2.0</td>
      <td>1900</td>
      <td>2150</td>
      <td>628</td>
      <td>9</td>
      <td>True</td>
      <td>False</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1.0</td>
      <td>1.0</td>
      <td>1903</td>
      <td>500</td>
      <td>1258</td>
      <td>9</td>
      <td>True</td>
      <td>False</td>
    </tr>
    <tr>
      <th>4</th>
      <td>0.0</td>
      <td>1.0</td>
      <td>1930</td>
      <td>500</td>
      <td>878</td>
      <td>10</td>
      <td>True</td>
      <td>False</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>487</th>
      <td>5.0</td>
      <td>2.5</td>
      <td>1890</td>
      <td>3073</td>
      <td>586</td>
      <td>76</td>
      <td>False</td>
      <td>True</td>
    </tr>
    <tr>
      <th>488</th>
      <td>2.0</td>
      <td>1.0</td>
      <td>1923</td>
      <td>1045</td>
      <td>665</td>
      <td>106</td>
      <td>False</td>
      <td>True</td>
    </tr>
    <tr>
      <th>489</th>
      <td>3.0</td>
      <td>2.0</td>
      <td>1922</td>
      <td>1483</td>
      <td>1113</td>
      <td>106</td>
      <td>False</td>
      <td>True</td>
    </tr>
    <tr>
      <th>490</th>
      <td>1.0</td>
      <td>1.0</td>
      <td>1983</td>
      <td>850</td>
      <td>764</td>
      <td>163</td>
      <td>False</td>
      <td>True</td>
    </tr>
    <tr>
      <th>491</th>
      <td>3.0</td>
      <td>2.0</td>
      <td>1956</td>
      <td>1305</td>
      <td>762</td>
      <td>216</td>
      <td>False</td>
      <td>True</td>
    </tr>
  </tbody>
</table>

Word2vec refers to a technique that can learn a word embedding (semantic representation) by guessing how to fill in the blank from the immediate context. Words that tend to appear in similar contexts probably have similar meanings, so an algorithm can learn the meaning of human words by examining a word's favorite neighbors. Words with similar neighbors should have similar representations. What word could appear in the following blank?

> The city of _________ has an oceanic climate.

This enables us to find synonyms as the [TensorFlow Embedding Projector](https://projector.tensorflow.org/) shows. But the authors also pointed out some interesting [examples of the learned relationships](https://arxiv.org/pdf/1301.3781.pdf#page=10), such as how, in the embedding space, `Paris - France + Italy = Rome`.

## Recurrent neural networks

Although word embeddings produce a semantic representation for the meaning of words, there still remains a challenge of how we might combine these word embeddings to form meaningful sentences. How might we use machine learning to combine word embeddings in a way that is sensitive to context?

Earlier, we learned two ways that neural networks could be used to classify handwritten digit images. But both approaches (scikit-learn `MLPClassifier` and Keras `Conv2D`) involved learning weights and biases only from the input pixel values. In language modeling where the possible sequences of words are infinite, it becomes much harder to train neural network weights and biases to directly handle every possibility.

**Recurrent neural networks** (RNNs) represent a different way of organizing a neural network by calculating the output of a neuron taking into account not only the inputs but also a *hidden state* that represents previously-generated outputs. Unlike *hidden layers* in a neural network, **hidden states** provide additional information about previously-generated steps to the current step. In other words, a RNN learns to predict the next word based on the current input as well as information obtained from previous words.

For example, we can use recurrent neural networks for language modeling by considering how they might generate a response sequence. Given an input `X` of one or more words, a recurrent neural network generates an output `O` one word at a time while considering the hidden state `V`.

![Recurrent neural network](https://upload.wikimedia.org/wikipedia/commons/b/b5/Recurrent_neural_network_unfold.svg)

Example code using keras: [character-level text generation with LSTM](https://keras.io/examples/generative/lstm_character_level_text_generation/)

### Sequence-to-sequence framework

The **seq2seq** framework utilizes 2 RNNs to implement "sequence-to-sequence" tasks:

- An **encoder** that learns a machine representation for an input sequence, or *context*.
- A **decoder** that can take a machine representation (context) and decode it to a target sequence.

Originally, these models were used for machine translation, so the problem was framed as [reading an input sentence "ABC" and producing "WXYZ" as output](https://arxiv.org/pdf/1409.3215.pdf#page=2). By changing the training set to give the encoder context and have the decoder predict the expected reply, [the seq2seq framework can be used to model conversations](https://arxiv.org/pdf/1506.05869.pdf#page=2). How does this framework differ from using a single recurrent neural network for language modeling?

Example code using keras: [character-level sequence-to-sequence modeling](https://keras.io/examples/nlp/lstm_seq2seq/). For word-level tasks, we can use word embeddings to provide more information to the model about the meaning of each word.

## Transformer architecture

With more data, the seq2seq approach can produce good results, but at a high computational cost because the hidden state needs to be updated on each step. Recurrent neural networks end up spending significant amounts of resources computing hidden states sequentially.

The **transformer architecture** addresses this by throwing out the RNN and instead utilizing a mechanism called *self-attention* to learn context without relying on hidden states. The goal of **self-attention** is to identify associations or relationships between different words in a sequence. Consider the sentence:

> The animal didn't cross the street because it was too tired

What does "it" refer to—the animal or the street? Let's read about why this makes a difference in translation on the [Google Research blog](https://blog.research.google/2017/08/transformer-novel-neural-network.html) on the seminal work ["Attention is all you need"](https://arxiv.org/abs/1706.03762) and study the [Tensor2Tensor Intro notebook](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb) to see for ourselves how attention heads learn different relationships between words.

Although the transformer architecture was not the first to introduce the idea of an attention mechanism, it has since become the most popular way to not only use attention but to define language models in general.